In [5]:
import torch

In [6]:
with open('../data/tinyshakespeare/input.txt','r',encoding='utf-8') as f:
    text = f.read()

In [7]:
print('Total rows in text',len(text))

Total rows in text 1115394


In [8]:
text[:1000]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [9]:
# build vocabulary table
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f'chars are: {''.join(chars)}')
print(f'vocab_size is: {vocab_size}')

chars are: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab_size is: 65


In [10]:
# build tokenizer
itos ={i:ch for i,ch in enumerate(chars)}
stoi ={ch:i for i,ch in enumerate(chars)}

encode = lambda s:[stoi[c] for c in s]
decode = lambda l:''.join([itos[i] for i in l])

In [11]:
print('hi there:',decode(encode('hi there')))

hi there: hi there


In [12]:
data = torch.tensor(encode(text),dtype=torch.long)
print(f'data shape and dtype are: {data.shape} {data.dtype}')
print(data[:10])

data shape and dtype are: torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])


In [13]:
# 将数据分割成训练集和测试集
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

In [14]:
# 上下文长度
block_size = 8
# 9个训练数据实际包含了8组训练数据
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [15]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input sequence is {context}, the target is {target}')

when input sequence is tensor([18]), the target is 47
when input sequence is tensor([18, 47]), the target is 56
when input sequence is tensor([18, 47, 56]), the target is 57
when input sequence is tensor([18, 47, 56, 57]), the target is 58
when input sequence is tensor([18, 47, 56, 57, 58]), the target is 1
when input sequence is tensor([18, 47, 56, 57, 58,  1]), the target is 15
when input sequence is tensor([18, 47, 56, 57, 58,  1, 15]), the target is 47
when input sequence is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is 58


In [16]:
# build batch，通过batch可以加速训练过程
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [17]:
xb,yb = get_batch('train')
print(f'x is:{xb}')
print(f'y is:{yb}')

print('-'*30)
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b][:t+1]
        target = yb[b][t]
        print(f'when input sequence is {context}, the target is {target}')

x is:tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
y is:tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
------------------------------
when input sequence is tensor([24]), the target is 43
when input sequence is tensor([24, 43]), the target is 58
when input sequence is tensor([24, 43, 58]), the target is 5
when input sequence is tensor([24, 43, 58,  5]), the target is 57
when input sequence is tensor([24, 43, 58,  5, 57]), the target is 1
when input sequence is tensor([24, 43, 58,  5, 57,  1]), the target is 46
when input sequence is tensor([24, 43, 58,  5, 57,  1, 46]), the target is 43
when input sequence is tensor([24, 43, 58,  5, 57,  1, 46, 43]), the target is 39
when input sequence is tensor([44]), the target is 53
when input sequence is te

In [18]:
print(xb) # our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [19]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
        
    def forward(self,idx,targets=None):
        logits = self.token_embedding_table(idx)  #B,T,C    
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            # 因为pytorch的cross_entropy函数要求输入(mini_batch,C)的格式，以bigram为例，C=vocab_size，实际有B*T个批次，所以需要将logits的形状变为(B*T,C)
            logits = logits.view(B*T,C)
            # targets也做相同的形状变换匹配输入格式
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
        
        return logits,loss
    
    def generate(self,idx,max_new_tokens):
        # idx是shape为（B,T）的输入，每个T时间窗内为token的index
        for _ in range(max_new_tokens):
            logits,loss = self(idx)
            # 取出最后一个时间步的输出作为输入的下一个token
            logits = logits[:,-1,:] # becomes shape (B,C)
            prob = F.softmax(logits,dim=-1) # becomes shape (B,C)
            # 随机采样一个token
            idx_next = torch.multinomial(prob,num_samples=1) # becomes shape (B,1)
            # 增加到序列中，作为下一次输入
            idx = torch.cat((idx,idx_next),dim=-1)
        return idx

In [20]:
m = BigramLanguageModel(vocab_size=vocab_size)
logits,loss = m(xb,yb)
print(logits.shape)

torch.Size([32, 65])


In [21]:
# 可以看到的loss是4.8786，但是我们可以预估初始的loss，应该是-ln(1/65)=4.1744。
loss

tensor(4.8786, grad_fn=<NllLossBackward0>)

In [22]:
start_idx = torch.zeros((1,1), dtype=torch.long)

In [23]:
print(decode(m.generate(idx=start_idx,max_new_tokens=100)[0].tolist()))


Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [24]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(),lr=1e-3)

In [25]:
batch_size = 32

for _ in range(10000):
    xb,yb = get_batch(split='train')
    
    logits,loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.5727508068084717


In [26]:
# 因为这些token之间还没有互相交流，所以仅仅依靠bigram模型生成的结果较差。
print(decode(m.generate(idx=start_idx,max_new_tokens=300)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht 


## The mathematic trick in self-attention

In [27]:
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channel
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [28]:
# 这里的主要想法是让token之间开始产生联系communication，产生联系的数字关系是求这些token在连续时间步上的平均值
# 计算平均值是一种很弱的计算联系的方式，因为直观上来看会丢失大量的信息
# 并且当前时间步的token的与其它token的关系计算方式是有讲究的：
# 1.当前token只能与前面时间步的token产生联系，即第5个时间步的token只能与第1，2，3，4个时间步的token产生联系，因此第5个时间步的token存储的信息是前4个时间步的token加上第5个时间步自己的信息的平均值；
# 2.第5个token不能与后续的token产生联系的原因是因为我们需要根据第5个时间步的token来预测第6个时间步的token，如果第6个时间步的token或者及后续的token也参与第5个token信息平均值的计算了，那么等于是泄露答案了；
# 3.这种注意力机制的设计是为了让模型根据历史的信息来实现未来数据的预测。

xbow = torch.zeros(B, T, C) # xbow代表的是词袋模型，通常代表数据的平均值
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # t,C
        xbow[b,t] = torch.mean(xprev, dim=0)

In [29]:
# 这里是聚合了历史token的当前token的信息值
# 可以看到在第1个batch中，第1个时间步的token和xbow信息值都等于[ 0.1808, -0.0700]
# 而在第2个时间步的token含有的信息[-0.0894, -0.4926]是通过计算，第1个时间步的token[ 0.1808, -0.0700]和第2个时间步的token[-0.3596, -0.9152]在T维度方向的平均值。
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [30]:
# 这里是原始token的值
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [31]:
# version 2
# 应用矩阵计算提高效率：
# 设置function矩阵wei,用来计算每个时间步及其之前时间步的加权求和值
wei = torch.tril(torch.ones(T, T)) # wei = weights 意为权重参数
wei = wei/wei.sum(dim=1,keepdim=True)
print(wei.shape) # 8,8
xbow_new = wei @ x # 8,8 @ 4,8,2 -> 4,8,2
print(xbow_new.shape) # 4,8,2
print(xbow_new[0])

torch.Size([8, 8])
torch.Size([4, 8, 2])
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


In [32]:
# 检查两个tensor是否逐元素逐位置相等
torch.allclose(xbow,xbow_new, rtol=1e-4, atol=1e-4)

True

In [33]:
# version 3
tril = torch.tril(torch.ones(T, T))
# 这里可以看成是初始各个时间步中的token之间的亲密度，初始为0，表示当前位置的token对其它时间步的token没有亲密度，
# 但是随着训练，这些token会逐渐发现对其它某些时间步的token感兴趣，因此它们的值不会一直恒定是0
wei = torch.zeros(T, T) 
# 通过下三角矩阵的mask遮蔽，保证了当前时间步的token只能从之前时间步的token中获取信息，而不能从之后的时间步获取信息
wei = wei.masked_fill(tril == 0, float('-inf'))
# 这里使用softmax将概率平均到每一个时间步的token上，使得每个时间步的token贡献权重都是相同的
wei = wei.softmax(dim=1)
xbow_3 = wei @ x # 8,8 @ 4,8,2 -> 4,8,2
torch.allclose(xbow_new,xbow_3) 

# 这里就是self-attention机制的preview,利用下三角遮蔽矩阵对过去时间步的token进行加权聚合，mask遮蔽矩阵会告诉我们，
# 当前时间步的token需要多少个历史token参与计算当前位置token的信息。

True

In [34]:
# version 4: self attention 单头自注意力机制的实现原理
torch.manual_seed(1337)
B,T,C = 4,8,32

tril = torch.tril(torch.ones(T, T))

# 单头注意力机制的计算过程
head_size = 16 #定义头数，其实是注意力模块最终输出的信息维度

# 0.x是注意力模块的输入信息
x = torch.randn(B,T,C)

# 1.注意力机制中每个token会发出两个向量，一个是query向量，一个是key向量
# 1.其中query向量是用来发出询问，我在查找什么内容？
# 1.key向量是用来存储，我包含了什么信息。
query = nn.Linear(C,head_size,bias=False)
key = nn.Linear(C,head_size,bias=False)

# value向量是最终将 query和key做点积之后的结果，做矩阵乘法，理解角度分析是，取出wei矩阵中隐藏的信息；物理意义是将输入x再转换成head头维度的结果
value = nn.Linear(C,head_size,bias=False)

# 1.q是 “我在查找什么？” 的计算矩阵
q = query(x) # B,T,head
# 1.k是 “我包含了什么信息？” 的计算矩阵
k = key(x) # B,T,head
# 2.wei是q和k的点积，存储的是不同的token通过对别的token进行询问，我在找什么，你包含什么，然后结果存储在wei中
# 2.wei矩阵相当于包含了private message，不同的token通过点积乘法，相似度高的在某个channel中得到了较大的数值，从而识别到了某些亲密度信息，意味着这两个token是有紧密联系的，类似向量的相似度计算使用点积
wei = q @ k.transpose(-1,-2) # B,T,T
# 2.对于每个token，其中包含的意义只能从历史token的信息中和自身包含的信息聚合而来，因此需要进行mask，将未来时间步的token的注意力权重置为负无穷
wei = wei.masked_fill(tril == 0, float('-inf'))
# 2.通过softmax，完成权重的归一化，将数值转换为0-1之间的概率分布，例如最后的结果中，第一个时间步的token包含的注意力权重就是1，因为只有它自己参与计算，它的权重就是1
#   而第二个时间步的权重来自于1，2两个位置的token，所以它们的权重会不同，但是softmax将他们的原始值量化到0-1之间，使得权重的总和为1
wei = wei.softmax(dim=-1) # B,T,T

# 3.v是token包含了什么结果的提取结果用计算矩阵
v = value(x) # B,T,head

# 4.使用v矩阵（提取token包含了什么的计算矩阵）和wei矩阵（token通过询问交流后携带了什么信息）进行矩阵乘法，取出这个单头注意力模块最终注意到的结果，是一个head维度的向量
out = wei @ v

#----------注意力机制的说明----------------
# 1.注意力机制是一种通讯机制 communication mechanism，它可以让信息在不同的token之间传递，并且可以对信息进行筛选，从而提取出有用的信息。
# 2.结合本章内的代码块《Block 1:利用矩阵规则来聚合信息》这个代码，可以理解到，通过mask_fill来遮蔽了未来时间步的token，
#   实际使得计算结果中的每个时间步的结果，都只采用当前时间步的token和历史token的信息进行计算当前t的信息。
#   这里结合《Block 1:利用矩阵规则来聚合信息》理解，视频参考《3blue1brown》的transformers视频关于token之间信息流动的方向，都是历史token和当前token指向当前时间步的这个位置最后计算得到聚合信息。
# 3.在计算过程中，实际每个token是不知道互相之间的空间位置，而合理的语言token是有特定的位置顺序的，因此需要编码其位置信息，作为输入信息一部分也参与计算。
# 4.由于mask_fill这种下三角函数的独特机制，单次计算就是一个下三角token矩阵，可以让模型在一次计算中看到很多个例子的感觉，逐渐学习到信息是如何被逐步增加，就像是一步步看到未来出现的token一样。
# 5.如果在某种情况下，你需要所有的token都进行通讯communicate交流，那么删除mask_fill这一行代码即可，这样模型会学习到所有的token之间的信息流动。
#   这种方式在语言翻译，情感分析中经常使用，被称之为encoder编码器，实现结果是将输入序列编码成固定长度的向量；
#   使用了mask_fill机制的被称为decoder解码器，因为decoder在预测后续的token，很像auto regressive的机制，这种行为像是在解码语言的规律。
#   注意力机制和编码器还是解码器无关，因为它仅仅代表一种通讯交流机制，注意力机制支持信息在任意节点之间的流动。
# 6.以上我们提到的都是自注意力机制（self-attention），这是因为我们构建的key query value矩阵都是从同一个源x产生的，这些token本质观察的都是同源信息，就像是这个query不仅会询问历史token的key，还会询问当前token的key一样。
#   还有交叉注意力机制（cross-attention），它的实现是query从x源产生，但是key和value矩阵是从另一个源y(外部信息)产生的，例如transformer架构中的encoder-decoder结构，query是decoder的输出，key和value是encoder的输出，
#   这种结构可以让decoder从encoder的输出中学习到不同于自身源x的信息，也就是形成交叉注意力。（但实际注意力机制还有更广泛的使用范围）
# 7.缩放注意力（scaling attention）是论文中提出的一种重要的normalization手段，主要目的是为了控制初始化的方差，参考本章《Why we need scaling attention?》，
#   因为假设当前k和q矩阵是均值为0方差为1的正态分布，那么矩阵乘法之后，结果矩阵wei的方差会乘head_size倍数的放大，越大的方差意味着矩阵内分布的数值越分散，极大值和极小值相差越大，
#   为什么较大的方差需要引起我们的关注，因为在后续的softmax计算过程中，如果数据存在一个较大的值，那么softmax会放大这个值，整体曲线朝这个值呈现锐化的分布，类似one-hot的结果，
#   这会导致在初始化学习的时候，这个token的注意力权重过大，从而无法从其它token中学习到更多的信息，这在初始化学习的时候是很不利的。

In [41]:
tril = torch.tril(torch.ones(block_size,block_size))

In [42]:
tril[:block_size,:block_size]

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [36]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5877, 0.4123, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4457, 0.2810, 0.2733, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2220, 0.7496, 0.0175, 0.0109, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0379, 0.0124, 0.0412, 0.0630, 0.8454, 0.0000, 0.0000, 0.0000],
        [0.5497, 0.2187, 0.0185, 0.0239, 0.1831, 0.0062, 0.0000, 0.0000],
        [0.2576, 0.0830, 0.0946, 0.0241, 0.1273, 0.3627, 0.0507, 0.0000],
        [0.0499, 0.1052, 0.0302, 0.0281, 0.1980, 0.2657, 0.1755, 0.1474]],
       grad_fn=<SelectBackward0>)

In [37]:
# Why we need scaling attention?
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)

wei_without_scaling = k @ q.transpose(-1,-2)
wei = k @ q.transpose(-1,-2) / head_size ** 0.5

print(f'the var of k is {k.var()}')
print(f'the var of q is {q.var()}')
print(f'the var of wei_without_scaling is {wei_without_scaling.var()}')
print(f'the var of wei is {wei.var()}')

the var of k is 1.044861912727356
the var of q is 1.0700464248657227
the var of wei_without_scaling is 17.46897315979004
the var of wei is 1.0918108224868774


In [38]:
# 上述方法可以实现，但是效率不高，高效的方式是使用矩阵乘法实现
# 矩阵乘法的优势在于可以并行计算，可以大大提高运算速度。
torch.manual_seed(42)

a = torch.ones(3,3)
b = torch.randint(0,10,(3,2)).float()

c = torch.matmul(a,b)

print(f'a=\n{a}')
print(f'b=\n{b}')
print(f'c=\n{c}')

# 这里可以看到矩阵c中每一行各列的值都分别是矩阵b中的对应列的所有行的元素之和，这里可以看作矩阵a是一个function矩阵，这个function的功能就是将矩阵b中的每一列的所有行元素求和。

a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])


In [39]:
# 如果对function矩阵进行变换，那么就会应用不同的计算规则对矩阵b进行计算

a = torch.tril(a)
c = torch.matmul(a,b)

print(f'a=\n{a}')
print(f'b=\n{b}')
print(f'c=\n{c}')

# 现在可以看到c的每一行的对应列结果都是b的对应行加上比其行索引小的行的对应列的结果

a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])


In [40]:
# Block 1:利用矩阵规则来聚合信息
# 为了解决之前计算效率不足的问题，我们可以构建新的function矩阵a

a = torch.tril(torch.ones(3, 3))
sum_a = torch.sum(a,dim=1,keepdim=True)
a = a/sum_a

c = torch.matmul(a,b)

print(f'a=\n{a}')
print(f'b=\n{b}')
print(f'c=\n{c}')

# 这样，无论怎么改变b的列数，都可以按照这种规则进行满足a这种function矩阵的扩展

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [43]:
class BatchNorm1d:
    def __init__(self,dim,eps=1e-5,momentum=0.001):
        self.eps = eps
        self.momentum = momentum
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        self.training = True
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)
        
    def __call__(self,x):
        if self.training:
            # batch mean and variance
            xmean = x.mean(0,keepdim=True)
            xvar = x.var(0,keepdim=True)    
        else:
            xmean = self.running_mean
            xvar = self.running_var
            
        # 动态计算总计样本的均值和方差
        if self.training:
            with torch.no_grad():
                self.running_mean = (1-self.momentum) * self.running_mean + self.momentum * xmean
                self.running_var = (1-self.momentum) * self.running_var + self.momentum * xvar
        
        xhat = (x-xmean)/torch.sqrt(xvar+self.eps)
        self.out = self.gamma * xhat + self.beta
        
        return self.out
    
    def parameters(self):
        return [self.gamma,self.beta]

In [ ]:
class LayerNorm1d:
    def __init__(self,dim,eps=1e-5):
        self.gamma = torch.ones(dim)  # (dim,)
        self.beta = torch.zeros(dim)
        self.eps = eps
        
    def __call__(self,x):
        # batch mean and variance
        xmean = x.mean(1,keepdim=True) # (batch,1)
        xvar = x.var(1,keepdim=True)   # (batch,1)

        xhat = (x-xmean)/torch.sqrt(xvar+self.eps)  # (batch,dim)
        self.out = self.gamma * xhat + self.beta  # (batch,dim)
        
        return self.out
    
    def parameters(self):
        return [self.gamma,self.beta]

In [ ]:
# batch norm测试
torch.manual_seed(1337)
module= BatchNorm1d(100)
x = torch.randn(32,100) # batch size=32, feature size=100
y = module(x)   # 经过norm层后输出的结果

In [45]:
print(y[:,0].mean(), y[:,0].var())

tensor(7.4506e-09) tensor(1.0000)


In [48]:
# layer norm测试
torch.manual_seed(1337)
module= LayerNorm1d(100)
x = torch.randn(32,100) # batch size=32, feature size=100
y = module(x)   # 经过norm层后输出的结果

In [49]:
print(y[0:,:].mean(), y[0,:].var())

tensor(-2.9802e-10) tensor(1.0000)
